In [1]:
import pandas as pd 
import collections
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import numpy as np

import math
from datetime import datetime 
from datetime import datetime, timedelta


from powernad.API.Campaign import *
from powernad.API.RelKwdStat import *

import time

from time import sleep
from urllib.error import HTTPError

import urllib.request
from datetime import datetime, timedelta
import json

from tqdm.notebook import tqdm

In [3]:
test = pd.read_csv('합침키워드.csv')

In [4]:
test.drop(['Unnamed: 0'], axis=1, inplace=True)

In [5]:
test.head()

,날짜,수집날짜,쿼리수,키워드
0,2021-03-20,2021-04-20,0.0,월드비전6k
1,2021-03-21,2021-04-20,0.0,월드비전6k
2,2021-03-22,2021-04-20,0.0,월드비전6k
3,2021-03-23,2021-04-20,0.0,월드비전6k
4,2021-03-24,2021-04-20,0.0,월드비전6k


In [6]:
lis = test['키워드'].values.tolist()

In [7]:
keyword_list=[]
for i in lis: 
    if i not in keyword_list:
        keyword_list.append(i)

In [8]:
len(keyword_list)

339

In [9]:
toda = datetime.now()
time_month = toda - relativedelta(months=1)
time_month = time_month.strftime('%Y-%m-%d')
time_month= str(time_month)


yesterday = toda - relativedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')
yesterday = str(yesterday)



today = str(datetime.now().date())

In [10]:
dt_index = pd.date_range(start=time_month, end = yesterday)


date = pd.DataFrame(data=dt_index, columns=['날짜'])
    
    

In [11]:
lis=[]
for i in tqdm(keyword_list) :
    test['날짜타입']=test['날짜'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d"))
    tmp = test[test['키워드'] == i]
    
    
    date_count = tmp['날짜'].tolist()
    date_count = (len(date_count)-1)
    
    for j in range(date_count) : 
        
        cnt = (tmp['날짜타입'].tolist()[j+1] - tmp['날짜타입'].tolist()[j]) - relativedelta(days=1)
        
        if str(cnt) == 'relativedelta()' :
            pass
        else:
            lis.append(i)
    
    
    
    
    
        

    

In [13]:
# 4월 20일  
#lis

[]

In [ ]:
#4월 21일 ->  어제 오류키워드에 더해서 나오는지 확인해보기
lis

In [17]:
#keyword_list = []
#for i in lis: 
#    if i not in keyword_list : 
#        keyword_list.append(i)

In [19]:
#keyword_list

['월드비전하이킹',
 '단기계약직',
 '굿네이버스토끼와거북이',
 '정기기부',
 '루이비통실버팔찌',
 '의식팔찌',
 '단기계약직실업급여',
 '3개월실업급여',
 '3개월계약직실업급여',
 '3개월근무실업급여',
 '1년계약직실업급여',
 '실업급여한달',
 '한달계약직실업급여',
 '마리몬드반지',
 '여름모자',
 'ccnmade',
 '뷔실팔찌',
 '방탄뷔팔찌',
 '최창남',
 '뷔비즈팔찌',
 '이에르로르',
 '이에르로르팔찌',
 'bts팔찌',
 '토끼와거북이',
 '남자불가리목걸이']

In [14]:
len(lis)

0

In [ ]:
#4월 20일 ->추가되는지 확인하기 

# 오류 키워드 

### 네이버 광고 API

In [15]:
BASE_URL = 'https://api.naver.com'
API_KEY = '********'
SECRET_KEY = '******'
CUSTOMER_ID = '*****'

In [16]:
rel = RelKwdStat(BASE_URL, API_KEY, SECRET_KEY, CUSTOMER_ID)

In [17]:
def search_keyword(searchword):
    

    kwdDataList = rel.get_rel_kwd_stat_list(siteId=None, biztpId=None, hintKeywords=searchword, event=None, month=None, showDetail='1')


    kwd_result = (kwdDataList[0].relKeyword, #키워드
                 kwdDataList[0].monthlyPcQcCnt, #월간 검색수 (PC)
                 kwdDataList[0].monthlyMobileQcCnt, # 월간 검색수 (Mobile)
                 kwdDataList[0].monthlyPcQcCnt+kwdDataList[0].monthlyMobileQcCnt) # 월간 total 

    return(kwd_result[3])

### 네이버 데이터랩 API

In [1]:
client_id = "****" 
client_secret = "***"

In [19]:
dictionary_yester={}
dictionary_now={}

In [20]:
for i in tqdm(lis):  
    sleep(1)
    try:
        if type(search_keyword(i)) !=str : 
            searchword = i.replace(" ","")
            a = search_keyword(searchword)


            url = "https://openapi.naver.com/v1/datalab/search"
            body = "{\"startDate\":\""+time_month+"\",\"endDate\":\""+today+"\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\""+i+"\",\"keywords\":[\""+i+"\"]}]}";
            requested = urllib.request.Request(url)
            requested.add_header("X-Naver-Client-Id", client_id)
            requested.add_header("X-Naver-Client-Secret", client_secret)
            requested.add_header("Content-Type", "application/json")
            response = urllib.request.urlopen(requested, data=body.encode("utf-8"))
            rescode = response.getcode()

            if(rescode==200):
                response_body = response.read()
                output_data = response_body.decode('utf-8')
            else:
                print('Error code:'+ rescode)
                pass

            result = json.loads(output_data)


            date = [a['period'] for a in result['results'][0]['data']]


            aa = pd.DataFrame({'date':date, 
                          i:[a['ratio'] for a in result['results'][0]['data']],
                          })

            # 일일 데이터 계산 


            sleep(0.5)
            total = aa[i].sum()


            aa[i] = aa[i].apply(lambda x :((x / total)*float(a)))

        else : 
            pass

    except (TypeError, IndexError,KeyError,ValueError):
        print(" 타입 or 인덱스 에러,Value 에러:",i)
        error.append(i)
        pass


    except HTTPError:
        print('http 에러:', i )
        error.append(i)
        pass

        #여기부터 새로코딩함 
    try: 
        if type(search_keyword(i)) !=str : 
            searchword = i.replace(" ","")
            a = search_keyword(searchword)        

            dt_index = pd.date_range(start=time_month, end= yesterday)
            dt_list = dt_index.strftime("%Y-%m-%d").tolist()

            date = pd.DataFrame(data=dt_list, columns=['날짜'])

            spred = pd.merge(date,aa,left_on='날짜',right_on='date',how='outer')
            spred.drop(['date'],inplace=True, axis=1)

            spred.replace(np.nan,0,inplace=True)


            a=[] 

            for j in range(len(spred)) : 
                a.append(today)


            spred['수집날짜'] = a

            spred = spred[['날짜','수집날짜',i]]

            if i in dictionary_yester:

                pass

            else : 
                dictionary_yester[i]=[]



            dictionary_now[i]=spred

        else:
            pass

    except (TypeError, IndexError,KeyError,ValueError):

        pass

    sleep(0.5)
    try :
        if type(search_keyword(i)) !=str : 
            searchword = i.replace(" ","")
            a = search_keyword(searchword)        

            if dictionary_yester[i]==[]:
                dictionary_yester[i] = dictionary_now[i]
               # for k in range(3):
                #    sleep(1)
                 #   gc2.append_row(dictionary_now[i].iloc[-1-k,:].values.tolist()+[i])


            else : 
                pass 
        else:
            pass


    except (TypeError, IndexError,KeyError):
          pass


    except ValueError:


            yester_df= dictionary_yester[i]
            today_df = dictionary_now[i]


            yester_df.set_index('날짜',inplace=True)

            today_df.set_index('날짜',inplace=True)

            yester_df.update(today_df)


            tmpt = today_df.iloc[-1,:]

            yester_df = yester_df.append(tmpt)

            yester_df.reset_index(inplace=True)
            today_df.reset_index(inplace=True)

            #dictionary_yester[i] = yester_df

        ## 구글 API로 데이터 올리기
            # 2주간의 데이터만 구글 스프레드 시트에 올리겠다.

          #  for k in range(3):
           #     sleep(1)
            #    gc2.append_row(yester_df.iloc[-1-k,:].values.tolist()+[i])




            dictionary_yester[i] = yester_df


    except :
        print('API 에러',i)
        error.append(i)



    sleep(3)


In [21]:
lis

[]

In [22]:
dictionary_now['롯데유니세프']

KeyError: '롯데유니세프'